<a href="https://colab.research.google.com/github/Harshkumar0403/AI-Text_Classification_using_bert/blob/main/model_using_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI Generated Text detection in Articles using BERT and Tensorflow**

![alt text](https://img.freepik.com/free-vector/brain-with-digital-circuit-programmer-with-laptop-machine-learning-artificial-intelligence-digital-brain-artificial-thinking-process-concept-vector-isolated-illustration_335657-2246.jpg?size=626&ext=jpg)

---


**Problem Statement**- Motivation with the rapid advancement of AI ,
distinguishing between human-written and AI generated content is
challenging. The task aim to accurately identify the origin of textual content , contributing to the development of robust techniques for detecting AI-generated content.


---

**Task -** Participants are task with developming model capable of distinguishing whether the text given was written by human or generated by AI tools , Given an article text, the goal is to classify its origin as human written or AI genarated .


In [ ]:
!pip install tensorflow_hub tensorflow_text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
raw_df=pd.read_csv("labelled_train_set.csv")
raw_df

,ID,Article,Type
0,TRAIN_1,There are a variety of emerging applications f...,Human-written
1,TRAIN_2,As each new means of communication and social ...,Human-written
2,TRAIN_3,"These suggestions include:, Learn about the pu...",Human-written
3,TRAIN_4,In recent years there has been growing concern...,Human-written
4,TRAIN_5,"By the late 1970s, researchers at many major u...",Human-written
...,...,...,...
495,TRAIN_496,Handwriting recognition technology has numerou...,AI-generated
496,TRAIN_497,Digital Rights Management (DRM) (sometimes als...,Human-written
497,TRAIN_498,A cookie is a small piece of data that is stor...,AI-generated
498,TRAIN_499,The hexadecimal system is a number system with...,AI-generated


In [ ]:
raw_df.groupby("Type").describe()

ID                      Article         \
              count unique      top freq   count unique   
Type                                                      
AI-generated    114    114  TRAIN_9    1     114    114   
Human-written   386    386  TRAIN_1    1     386    385   

                                                                       
                                                             top freq  
Type                                                                   
AI-generated   The idea of networks has been around for centu...    1  
Human-written  Starting in the late 1950s, in computer facili...    2

In [ ]:
raw_df['Type'] = raw_df['Type'].apply(lambda x: 1 if x=='AI-generated' else 0)
raw_df

,ID,Article,Type
0,TRAIN_1,There are a variety of emerging applications f...,0
1,TRAIN_2,As each new means of communication and social ...,0
2,TRAIN_3,"These suggestions include:, Learn about the pu...",0
3,TRAIN_4,In recent years there has been growing concern...,0
4,TRAIN_5,"By the late 1970s, researchers at many major u...",0
...,...,...,...
495,TRAIN_496,Handwriting recognition technology has numerou...,1
496,TRAIN_497,Digital Rights Management (DRM) (sometimes als...,0
497,TRAIN_498,A cookie is a small piece of data that is stor...,1
498,TRAIN_499,The hexadecimal system is a number system with...,1


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_bert_embeddings(texts):
    # Preprocess the input text
    preprocessed_text = bert_preprocess(texts)

    # Pass through BERT encoder to get embeddings
    embeddings = bert_encoder(preprocessed_text)['pooled_output']  # CLS token

    # If using eager execution, convert to NumPy
    if tf.executing_eagerly():
        return embeddings.numpy()  # Convert to NumPy in eager mode
    else:
        return tf.make_ndarray(embeddings)  # Handle graph mode case

# Ensure eager execution (TensorFlow 2.x has it enabled by default)
tf.config.run_functions_eagerly(True)




In [ ]:
articles = raw_df['Article'].tolist()
types = raw_df['Type'].tolist()

In [ ]:
bert_embeddings = get_bert_embeddings(articles)

# Create a new DataFrame to store embeddings along with 'Type'
embedding_df = pd.DataFrame(bert_embeddings)
embedding_df['Type'] = types

In [ ]:


# Show the new DataFrame
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Type
0,-0.761111,-0.497488,-0.976464,0.502865,0.847911,-0.541581,0.119783,0.337718,-0.878875,-0.999947,...,-0.600982,0.978982,0.454468,-0.647529,-0.204099,0.698039,-0.948280,-0.739069,0.577836,0
1,-0.748050,-0.751882,-0.993540,0.769780,0.949255,-0.649007,0.521343,0.479075,-0.947622,-0.999970,...,-0.468695,0.998210,0.220175,-0.948662,-0.709448,0.819729,-0.990678,-0.769989,0.488416,0
2,-0.800765,-0.672275,-0.996831,0.830917,0.945472,-0.415503,0.650205,0.434568,-0.986296,-0.999995,...,-0.676413,0.995003,0.527899,-0.817697,-0.848318,0.916039,-0.976991,-0.757809,0.663629,0
3,-0.846199,-0.671000,-0.943549,0.493270,0.821896,-0.595142,0.043064,0.571923,-0.877375,-0.999967,...,-0.505888,0.982417,0.533950,-0.824722,-0.178963,0.759079,-0.911759,-0.772329,0.402170,0
4,-0.428321,-0.402226,-0.849490,-0.064152,0.778023,-0.239085,-0.594790,0.191015,-0.473263,-0.998958,...,-0.471794,0.935291,0.082402,-0.393136,0.475093,0.517276,-0.935621,-0.405051,0.124229,0


In [ ]:
embedding_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Columns: 769 entries, 0 to Type
dtypes: float32(768), int64(1)
memory usage: 1.5 MB


In [ ]:
from sklearn.utils import resample

# Separate the majority and minority classes
df_majority = embedding_df[embedding_df['Type'] == 0]  # Human-written
df_minority = embedding_df[embedding_df['Type'] == 1]  # AI-generated

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # Sample with replacement
                                 n_samples=len(df_majority),    # Match the number of majority class
                                 random_state=42)  # Reproducible results

# Combine majority class with upsampled minority class
upsampled_df = pd.concat([df_majority, df_minority_upsampled])

# Shuffle the combined dataframe
upsampled_df = upsampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print(upsampled_df['Type'].value_counts())

Type
1    386
0    386
Name: count, dtype: int64


In [ ]:
# Extract embeddings and labels
X = upsampled_df.drop(['Type'], axis=1).values  # Convert DataFrame to numpy array
y = upsampled_df['Type'].values  # Convert labels to numpy array

# Convert numpy arrays to tensors
X = tf.convert_to_tensor(X, dtype=tf.float32)  # Ensure dtype matches your model's input
y = tf.convert_to_tensor(y, dtype=tf.float32)

# Check the shapes of X and y
print(X.shape)
print(y.shape)

(772, 768)
(772,)


In [ ]:
from sklearn.model_selection import train_test_split

# Convert to numpy arrays if not already
X = X.numpy() if isinstance(X, tf.Tensor) else X
y = y.numpy() if isinstance(y, tf.Tensor) else y

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert to tensors if needed
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

In [ ]:
import tensorflow as tf

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),  # Input layer with the shape of the BERT embeddings
    tf.keras.layers.Dropout(0.1),          # Dropout layer for regularization
    tf.keras.layers.Dense(64, activation='relu'),  # Hidden dense layer with ReLU activation
    tf.keras.layers.Dropout(0.1),          # Another dropout layer
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification
              metrics=METRICS)

# Show the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dropout (Dropout)                    │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          49,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,281 (192.50 KB)

 Trainable params: 49,281 (192.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.5885 - loss: 0.7164 - precision: 0.5521 - recall: 0.5310
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.7452 - loss: 0.5302 - precision: 0.7853 - recall: 0.6787
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.8278 - loss: 0.4230 - precision: 0.7904 - recall: 0.8648
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.8584 - loss: 0.3528 - precision: 0.8687 - recall: 0.8556
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.8541 - loss: 0.3306 - precision: 0.8698 - recall: 0.8377
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.8842 - loss: 0.2981 - precision: 0.8521 - recall: 0.9118
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.8905 - loss: 0.2721 - precision: 0.9031 - recall: 0.8738
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.9033 - loss: 0.2440 - precision: 0.9052 - recall: 0.9091
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━

In [ ]:
model.evaluate(X_val, y_val)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8827 - loss: 0.2516 - precision: 0.8360 - recall: 0.9743


[0.2548539936542511,
 0.8903225660324097,
 0.8510638475418091,
 0.9638554453849792]

In [ ]:
test_df = pd.read_csv("unlabelled_test2.csv")
test_df

,ID,Article
0,TEST_1,Grid computing is a type of distributed comput...
1,TEST_2,"RAID stands for ""Redundant Array of Independen..."
2,TEST_3,A computer password is a sequence of character...
3,TEST_4,For people who find commercial online services...
4,TEST_5,A Green PC is a computer that has been designe...
...,...,...
512,TEST_513,A Computer Security Incident Response Team (CS...
513,TEST_514,"Recordable CDs, also known as CD-Rs, are a typ..."
514,TEST_515,A webmaster is a person responsible for managi...
515,TEST_516,The standard for serial transmission is a set ...


In [ ]:
embeddings_test = get_bert_embeddings(test_df['Article'].tolist())
embeddings_test.shape

(517, 768)

In [ ]:
predictions = model.predict(embeddings_test)

# Convert predictions to binary labels (0 or 1)
predicted_labels = (predictions > 0.4).astype(int).flatten()

 9/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [ ]:
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'Prediction': predicted_labels
})

# Save to CSV
submission_df.to_csv('submission1.csv', index=False)

In [ ]:
submit_df= pd.read_csv('submission1 (4).csv')
submit_df

,ID,Prediction
0,TEST_1,1
1,TEST_2,1
2,TEST_3,1
3,TEST_4,1
4,TEST_5,1
...,...,...
512,TEST_513,1
513,TEST_514,1
514,TEST_515,1
515,TEST_516,1


In [ ]:
pd.value_counts(submit_df['Prediction'])

<ipython-input-9-63ac3a168340>:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(submit_df['Prediction'])


,count
Prediction,
1,474
0,43


# **Result**

Overall the model gives 0.881 macF1 score for the test data.